In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook

In [2]:
dtype = torch.float
device = torch.device('cuda:0')

In [ ]:
torch.tensor([5.5,3])

In [ ]:
N, D_in, H , D_out = 64,1000, 100, 10

## How to implement neural network using Pytorch

* import torch
* define datatype and device
* define input shape and produce or load inputs
* define hyperparameters
* start loop for n no. of iterations
* compute y_pred using model
* compute loss using loss function
* Use automatic differentiation to compute derivatives
* Update the parameters using optimizer

# Implementing Simple neural network with pytorch

In [ ]:
x = torch.randn(N,D_in, device = device, dtype = dtype)
y = torch.randn(N,D_out, device = device, dtype = dtype)

w1 = torch.randn(D_in,H, device = device, dtype = dtype)
w2 = torch.randn(H, D_out, device = device, dtype = dtype)

learning_rate = 1e-6

for t in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    loss = (y_pred - y).pow(2).sum().item()
    print(t,loss)
    
    grad_y_pred = 2.0 * (y_pred- y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h< 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    w1 -= learning_rate*grad_w1
    w2 -= learning_rate*grad_w2   

# Pytorch with Automatic differentiation

In [ ]:
#with automatic differentiation
N, D_in, H, D_out = 64, 1000,100,10

x = torch.randn(N,D_in, dtype = dtype, device = device)
y = torch.randn(N,D_out, dtype = dtype, device = device)

w1 = torch.randn(D_in,H, 
                 dtype = dtype, 
                 device = device, 
                 requires_grad=True)
w2 = torch.randn(H,D_out, 
                 dtype = dtype, 
                 device = device,
                 requires_grad= True)

learning_rate= 1e-6

for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()
        

# Defining custom autograd functions 

In [ ]:
#define own autograd functions

class MyReLU(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx,grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input< 0] = 0
        return grad_input

dtype = torch.float
device = torch.device('cuda:0')

N, D_in, H, D_out = 64,1000,100,10

x = torch.randn(N,D_in, device = device, dtype = dtype)
y = torch.randn(N, D_out, device= device, dtype = dtype)

w1 = torch.randn(D_in, H, device = device , dtype= dtype,
                requires_grad = True)
w2 = torch.randn(H, D_out, device= device, dtype = dtype,
                requires_grad= True)

learning_rate = 1e-6

In [ ]:
for t in range(500):
    relu = MyReLU.apply
    
    y_pred = relu(x.mm(w1)).mm(w2)
    
    loss =(y_pred- y).pow(2).sum()
    print(t, loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

# Using NN module (creating model)

In [ ]:
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),
    
)

loss_fn = torch.nn.MSELoss(size_average=False)

learning_rate = 1e-4
for t in range(400):
    
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# Using Optimizer for automatic weight updation

In [ ]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(size_average=False)

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),learning_rate)

for i in range(400):
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    
    print(t, loss.item())
    
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

# Creating a custom neural network

In [ ]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in,H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self,x):
        h_relu = self.linear1(x).clamp(min= 0)
        y_pred = self.linear2(h_relu)
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = TwoLayerNet(D_in, H,D_out)

criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), 1e-4)

for i in range(100):
    y_pred = model.forward(x)
    
    loss= criterion(y_pred,y)
    print(i, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

# CNN Using Pytorch

In [ ]:
class Net(nn.Module):
    def __init__(self):
        #initializing nn module 
        super(Net,self).__init__()
        
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        #to reshape the tensor
        #passing -1 tells this function to compute the no. of rows for us
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *=s
        return num_features
    
net = Net()
print(net)

In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

In [ ]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out.shape)

# CIFAR10 using Pytorch 

In [3]:
import torch 
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root = './data', train= True,
                                       download= True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4,
                                         shuffle = True, num_workers = 2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download= True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 4,
                                        shuffle = True, num_workers = 2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse',
          'ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    print(npimg.shape)
    print(np.transpose(npimg,(1,2,0)).shape)
    plt.imshow(np.transpose(npimg, (1,2,0)))

dataiter = iter(trainloader)
images , labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
#         self.conv4 = nn.Conv2d(128,256,5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128*12*12, 512)
        self.fc2 = nn.Linear(512,124)
        self.fc3 = nn.Linear(124,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))
        
#         print(x.shape)
        x = x.view(-1,128*12*12)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)


In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i,data in enumerate(tqdm_notebook(trainloader,0)):
        inputs, labels = data
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i%2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                 (epoch +1, i+1, running_loss/ 2000))
            running_loss = 0.0
            

print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth:',' '.join('%5s' % classes[labels[j]] for j in range(4)))


In [ ]:
images = images.to(device)
output = net(images)

# output = output.to(device)
_, predicted = torch.max(output,1)

print('predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        images,labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct/ total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        outputs = outputs.to(device)
        _,predicted = torch.max(outputs,1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
for i in range(10):
    print('Accuracy of %5s: %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

# Transfer learning using Pytorch 

In [ ]:
model_resnet = torchvision.models.resnet18(pretrained=True)

num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs,10)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr = 0.01)



In [ ]:
model_resnet.cuda()

In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i,data in enumerate(tqdm_notebook(trainloader,0)):
        inputs, labels = data
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model_resnet(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i%2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                 (epoch +1, i+1, running_loss/ 2000))
            running_loss = 0.0
            

print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth:',' '.join('%5s' % classes[labels[j]] for j in range(4)))

images = images.to(device)
output = model_resnet(images)

# output = output.to(device)
_, predicted = torch.max(output,1)

print('predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        images,labels = images.to(device), labels.to(device)
        outputs = model_resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct/ total))

# Loading Font data


In [5]:
import torch
from torchvision import transforms,datasets

transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

train_dir = datasets.ImageFolder(root = 'D:/Work/fastai/courses/dl1/data/English/train',
                                transform = transform)
train_dataset_loader = torch.utils.data.DataLoader(train_dir,
                                            batch_size = 64, shuffle= True,
                                            num_workers=4)
test_dir = datasets.ImageFolder(root = 'D:/Work/fastai/courses/dl1/data/English/valid',
                               transform = transform)

test_dataset_loader = torch.utils.data.DataLoader(test_dir,
                                                 batch_size =64, shuffle= True,
                                                 num_workers= 4)

In [6]:
import matplotlib.pyplot as plt
import numpy as np

def image_show(img):
    img = img/2 + 0.5
    img = img.numpy()
    plt.imshow(np.transpose(img,(1,2,0)))
    
dataiter = iter(dataset_loader)
images,labels = dataiter.next()

image_show(torchvision.utils.make_grid(images))


NameError: name 'dataset_loader' is not defined

In [7]:
model_resnet= torchvision.models.resnet18(pretrained=True)
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features,62)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(),lr = 0.01)
model_resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [8]:
for epoch in range(2):
    running_loss = 0.0
    for i,data in enumerate(tqdm_notebook(train_dataset_loader,0)):
        inputs,label = data
        inputs,label = inputs.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model_resnet(inputs)
        loss = criterion(outputs,label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 99:
            print('iteration: %d, loss: %.3f' % (i+1, running_loss/100))
            running_loss = 0.0
        
        
        

KeyboardInterrupt: 

In [ ]:
dataiter = iter(test_dataset_loader)
images, labels = dataiter.next()
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    print(npimg.shape)
    print(np.transpose(npimg,(1,2,0)).shape)
    plt.imshow(np.transpose(npimg, (1,2,0)))


# imshow(torchvision.utils.make_grid(images))
print('GroundTruth:',' '.join('%5s' % labels[j] for j in range(4)))

images = images.to(device)
output = model_resnet(images)

# output = output.to(device)
_, predicted = torch.max(output,1)
print('predicted: ', ' '.join('%5s' % predicted[j] for j in range(4)))